<a href="https://colab.research.google.com/github/janishjindal/Google-Colab-Notebooks/blob/main/TrainModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
from google.colab import drive
import pandas as pd
import pickle
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/AffableTrainData/Train.csv')

Mounted at /content/drive


Feature expansion from Col ImageLabels (isPerson)

In [ ]:
personFeatures = {'neck', 'hair', 'eyelash', 'beard', 'human leg', 'face', 'facial expression', 'lip', 'chin', 'moustache', 'blond', 'human body', 'arm', 'human', 'waist', 'man', 'woman', 'wrist', 'finger', 'nose', 'person', 'leg', 'cheek', 'tongue', 'lady', 'muscle', 'forehead', 'shoulder', 'eye', 'boy', 'mouth', 'male', 'girl', 'skin', 'thigh', 'smile', 'chest', 'nail','head', 'hand', 'people', 'eyebrow', 'layered hair', 'hair coloring', 'brown hair', 'facial hair', 'hair accessory', 'thumb', 'jaw', 'hip'}

In [ ]:
import ast
total_rows = len(df)
isPerson = []
confidence_threshold = 0.65
facialFeature_condifence = 0.0
newDfColumn = []
for row in range(total_rows):
  try:
    imageLabelList = ast.literal_eval(df.imageLabels[row])
    newImageLabelList = []
    for tup in imageLabelList :
      if(tup[1] >= confidence_threshold):
        newImageLabelList.append(tup)
    pf_count = 0
    for tup in newImageLabelList:
      if(tup[0] in personFeatures):
        pf_count+=1
    pf_percent = pf_count/len(newImageLabelList)
    newDfColumn.append(newImageLabelList)
    if(pf_percent > facialFeature_condifence):
      isPerson.append(1)
    else:
      isPerson.append(0)
  except ValueError:
    #print("Encountered null")
    newDfColumn.append([])
    isPerson.append(0)
    continue
  except ZeroDivisionError:
    # newImageList is empty => no image label with high confidence tag
    newDfColumn.append([])
    isPerson.append(0)
    continue
df["newImageLabels"] = newDfColumn
df["isPerson"] = isPerson

Bag of Word (BoW) on ImageLabel tags

In [ ]:
imageLabelBoWCol = []
for row in range(total_rows):
  try:
    imageLabelList = ast.literal_eval(df.imageLabels[row])
    bow = ""
    for tup in imageLabelList :
      if(tup[1] > confidence_threshold):
        bow += tup[0] + " " 
    imageLabelBoWCol.append(bow)    
  except ValueError:
    #print("Encountered null")
    imageLabelBoWCol.append("")
    continue
df["imageLabelBoW"] = imageLabelBoWCol

In [ ]:
df_orig=df.copy()

Feature Selection : Removing unwanted cols based on domain knowledge

In [ ]:
df.drop(['biography', 'username','externalUrl', 'picture', 'cityId', 'latitude', 'longitude', 'zip', 'contactPhoneNumber', 'imageLabels', 'newImageLabels'], axis = 1, inplace=True)

Applying CountVectorizer on ImageLabelBoW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
cv.fit(df['imageLabelBoW'])
cv_fit = cv.transform(df['imageLabelBoW'])
bow_df = pd.DataFrame(cv_fit.toarray(),columns=cv.get_feature_names_out())
bow_df.rename(columns = {'brand':'brand_bow'}, inplace = True)
#bow_df.info()

with open('/content/drive/My Drive/AffableModels/cv_pkl.pickle', 'wb') as f:
    pickle.dump(cv, f)

Handling Missing Values

In [ ]:
numerical_vars = ['mediaCount', 'followingCount', 'followerCount', 'usertagsCount', 'geoMediaCount', 'shoppablePostsCount', 'followingTagCount']
with open('/content/drive/My Drive/AffableModels/df_train_pkl.pickle', 'wb') as f:
    pickle.dump(df, f)
    
for i in numerical_vars:
  df[i].fillna(df[i].mean(), inplace=True)

categorical_vars = ['isBusiness', 'category', 'hasEmail', 'isPerson']
for i in categorical_vars:
  df[i].fillna(df[i].mode()[0], inplace=True)

Concatenating CountVectorizer of ImageLabels with Original df 

In [ ]:
df = pd.concat([df,bow_df], axis=1)
df.drop('imageLabelBoW', axis=1, inplace=True)

One-hot encoding for categical vars

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categorical_vars = ['isBusiness', 'category', 'hasEmail', 'isPerson']

category_encoder = OneHotEncoder(handle_unknown="ignore")
category_encoder.fit(df[['category']])
category_array = category_encoder.transform(df[['category']]).toarray()
category_ohe_df = pd.DataFrame(category_array, columns=category_encoder.get_feature_names_out(['category']))

isBusiness_encoder = OneHotEncoder(handle_unknown="ignore")
isBusiness_encoder.fit(df[['isBusiness']])
isBusiness_array = isBusiness_encoder.transform(df[['isBusiness']]).toarray()
isBusiness_ohe_df = pd.DataFrame(isBusiness_array, columns=isBusiness_encoder.get_feature_names_out(['isBusiness']))


hasEmail_encoder = OneHotEncoder(handle_unknown="ignore")
hasEmail_encoder.fit(df[['hasEmail']])
hasEmail_array = hasEmail_encoder.transform(df[['hasEmail']]).toarray()
hasEmail_ohe_df = pd.DataFrame(hasEmail_array, columns=hasEmail_encoder.get_feature_names_out(['hasEmail']))

isPerson_encoder = OneHotEncoder(handle_unknown="ignore")
isPerson_encoder.fit(df[['isPerson']])
isPerson_array = isPerson_encoder.transform(df[['isPerson']]).toarray()
isPerson_ohe_df = pd.DataFrame(isPerson_array, columns=isPerson_encoder.get_feature_names_out(['isPerson']))

ohe_df = pd.concat([category_ohe_df, isBusiness_ohe_df, hasEmail_ohe_df, isPerson_ohe_df], axis=1)
df = pd.concat([df, ohe_df], axis=1)
df.drop(categorical_vars, axis=1, inplace=True)

with open('/content/drive/My Drive/AffableModels/category_ohe_pkl.pickle', 'wb') as f:
    pickle.dump(category_encoder, f)
with open('/content/drive/My Drive/AffableModels/isBusiness_ohe_pkl.pickle', 'wb') as f:
    pickle.dump(isBusiness_encoder, f)
with open('/content/drive/My Drive/AffableModels/hasEmail_ohe_pkl.pickle', 'wb') as f:
    pickle.dump(hasEmail_encoder, f)
with open('/content/drive/My Drive/AffableModels/isPerson_ohe_pkl.pickle', 'wb') as f:
    pickle.dump(isPerson_encoder, f)


Training ML Models (Original + No Scaling + No SVD) -> Best

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score
from xgboost import XGBClassifier

x_train = df[df.columns.drop(['brand'])]
y_train = df['brand']

model=XGBClassifier(n_estimators=200,learning_rate=.2,random_state=201)

model.fit(x_train, y_train)

with open('/content/drive/My Drive/AffableModels/trainXGB_pkl.pickle', 'wb') as f:
    pickle.dump(model, f)
    
drive.flush_and_unmount()

In [ ]:
#drive.mount('/content/drive')
model.save_model("/content/drive/My Drive/AffableModels/XGBoost.bin")

xgboost.sklearn.XGBClassifier